In [ ]:
import pandas as pd
import re
from datetime import datetime

# ── Load Data ──────────────────────────────────────────────────────────────────
# Fixed: Using a raw string (r'') to avoid Unicode escape errors with backslashes
file_path = r'C:\Users\MubarakTijani\Data_governance\customers_raw.csv'
df = pd.read_csv(file_path, dtype=str)  # load everything as string so we see raw values
df.columns = df.columns.str.strip()               # remove accidental spaces in column names

TOTAL_ROWS = len(df)
print(f"Loaded {TOTAL_ROWS} rows, {len(df.columns)} columns\n")

report_lines = []

def add(text=""):
    report_lines.append(text)
    print(text)

# ── Data Quality Checks ────────────────────────────────────────────────────────
add("### Data Quality Summary")
add(f"Total Rows: {TOTAL_ROWS}")

if TOTAL_ROWS > 0:
    # 1. Missing Values
    null_counts = df.isnull().sum()
    add("\nMissing Values per Column:")
    add(null_counts.to_string())

    # 2. Duplicate Rows
    dup_count = df.duplicated().sum()
    add(f"\nDuplicate Rows: {dup_count} ({(dup_count/TOTAL_ROWS)*100:.2f}%)")

    # 3. Unique Values per Column
    add("\nUnique Values per Column:")
    for col in df.columns:
        add(f"{col}: {df[col].nunique()} unique values")
else:
    add("\nNotice: The dataset is empty.")
